In [2]:
%reset -f 
%whos

Interactive namespace is empty.


In [ ]:
#only run this cell once in anaconda command line to enable use of uobrainflex python environment for this jupyter notebook
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 21 12:19:27 2022
@author: Evan Vickers
this notebook loads and preprocesses mesoscope data within the uobrainflex environment
"""

python -m ipykernel install --user --name=uobrainflex

In [99]:
#%reset
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

#Misc
import time, os, sys, pdb
from glob import glob
from fnmatch import fnmatch

#Base
import numpy as np
import pandas as pd
import math as m
import scipy.io as sio
import scipy.stats as st
from scipy.ndimage import gaussian_filter1d
from scipy.stats import zscore

#Save
import json
import scipy.io as sio
import h5py

#Plot
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import cv2
from PIL import Image
from matplotlib import cm
from matplotlib.backends.backend_pdf import PdfPages
sns.set_style("ticks")

#Model
import ssm

#CCM
#from DelayEmbedding import DelayEmbedding as DE

#User
#import util
#import plotting as usrplt

import warnings
warnings.filterwarnings("ignore")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
PlotDir = os.path.join('I:\\uobrainflex_analysis_2022\\meso_sci_ppr_I_2022\\plots',fsuffix)
#PlotDir='I:\uobrainflex_analysis_2022\meso_sci_ppr_I_2022\plots\3056_200924_E235_1_00003_00001'
print(PlotDir)

I:\uobrainflex_analysis_2022\meso_sci_ppr_I_2022\plots\3056_200821_E235_1_00001_00001
3056_200821_E235_1_00001_00001
I:\uobrainflex_analysis_2022\meso_sci_ppr_I_2022\plots\3056_200821_E235_1_00001_00001


In [29]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 16 11:30:51 2021
@author: admin
"""

import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage.measure import block_reduce
from skimage.transform import PiecewiseAffineTransform, warp
from skimage.filters import gaussian
from scipy import ndimage
import scipy.ndimage.morphology as ni

def import_multi_tif(tif_folder, n_channels=2, down_sample_factor= 2):
    tif_filepaths = glob.glob(os.path.join(tif_folder + '\*.tif'))
    print('\n' + str(len(tif_filepaths)) + ' file(s) to process')
    for i, image_path in enumerate(tif_filepaths):
        im = io.imread(image_path)
        if n_channels==2:
            if i == 0:
                wf_blue = np.ndarray([0, int(im.shape[1]/down_sample_factor), int(im.shape[2]/down_sample_factor)])
                wf_green = np.ndarray([0, int(im.shape[1]/down_sample_factor), int(im.shape[2]/down_sample_factor)])
                
            first = np.atleast_3d(block_reduce(im[np.arange(0,len(im),2)],(1,down_sample_factor,down_sample_factor),func=np.mean))
            second = np.atleast_3d(block_reduce(im[np.arange(1,len(im),2)],(1,down_sample_factor,down_sample_factor),func=np.mean))
            if wf_blue.shape[0] == wf_green.shape[0]:
                wf_blue = np.append(wf_blue, first, axis=0)
                wf_green = np.append(wf_green, second, axis=0)
            else:
                wf_blue = np.append(wf_blue, second, axis=0)
                wf_green = np.append(wf_green, first, axis=0)                  
        print('\nFile ' + str(i+1) + ' of ' + str(len(tif_filepaths)) + ' complete')
    return wf_blue, wf_green

def set_transform_anchors(stationary_image, warp_image):
    figure = plt.figure()
    ax = plt.subplot(1,2,1)
    ax.imshow(stationary_image)
    ax2 = plt.subplot(1,2,2)
    ax2.imshow(warp_image)
    
    plt.suptitle('Set points to anchor transformation\n'+
                 'Select point on left then right image for each anchor\n'+
                 'left click to select | right click to cancel last select | middle click to complete')
    points = np.array(plt.ginput(100,timeout=300))

    stationary_points = points[np.array(range(0,len(points),2),dtype=int)]
    warp_points = points[np.array(range(1,len(points),2),dtype=int)]
    ax.scatter(stationary_points[:,0],stationary_points[:,1],color='b')
    for idx in range(len(stationary_points)):
        ax.text(stationary_points[idx][0],stationary_points[idx][1],str(idx),color='r')
    ax2.scatter(warp_points[:,0],warp_points[:,1],color='b')
    for idx in range(len(warp_points)):
        ax2.text(warp_points[idx][0],warp_points[idx][1],str(idx),color='r')
    plt.pause(0.1)
    return stationary_points, warp_points, figure

def transform_image(stationary_image,warp_image,stationary_points, warp_points):
    tform = PiecewiseAffineTransform()
    tform.estimate(stationary_points, warp_points)
    out_rows = stationary_image.shape[0]
    out_cols = stationary_image.shape[1]
    warped_image = warp(warp_image, tform, output_shape=(out_rows, out_cols))
    return warped_image

def smooth_masks(masks):
    for i in range(masks.shape[0]):
        img = masks[i,:,:]
        dst = gaussian(1-img,sigma=(1))
        dst[dst<=.01]=1
        dst[dst<.1]=0
        masks[i,:,:]=dst
    return masks

def reduce_mask_overlap(masks):
    for i in range(masks.shape[0]):
        this_mask = masks[i,:,:]
        other_masks = masks[np.where([np.arange(masks.shape[0])!=i])[1],:,:]
        other_masks_merge = np.max(other_masks,axis=0)
        overlap = np.where((this_mask+other_masks_merge)>1)
        this_mask[overlap[0],overlap[1]]=0
        masks[i,:,:]=this_mask
    return masks

def masks_to_outlines(masks):
    outlines = np.zeros(masks.shape,dtype=np.int8)
    for i in range(outlines.shape[0]):
        mask = masks[i,:,:]
        outlines[i,:,:] = mask - ni.binary_erosion(mask)
    return outlines

def rotate_3d_matrix(array_3d,angle = 22.5):   
    this_slice = array_3d[0,:,:]
    this_rotated = ndimage.rotate(this_slice, angle, reshape=True)
    rotated_matrix = np.ndarray([array_3d.shape[0],this_rotated.shape[0],this_rotated.shape[1]],dtype=np.int8)
    for idx in range(array_3d.shape[0]):
        this_slice = array_3d[idx,:,:]
        if len(np.where(this_slice==1)[0])>0:
            this_rotated = ndimage.rotate(this_slice, angle, reshape=True)
            this_rotated[this_rotated>=.2]=1
            this_rotated[this_rotated<.2]=0
            rotated_matrix[idx,:,:] = this_rotated
    return rotated_matrix

def surface_projection(volume):
    shell_volume = np.zeros(volume.shape,np.int8)
    for ii in range(volume.shape[0]):
        for jj in range(volume.shape[2]):
            zplane = np.where(volume[ii,:,jj]>0)[0]
            if len(zplane)>0:
                shell_volume[ii,zplane[0],jj] = 1
    return shell_volume

In [31]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 16 17:09:27 2021
@author: Daniel Hulsey
this script is used to align a CCF to a multimodalmapping session. Points selected to align the ccf will be saved for future use.
"""

import os
from skimage import io
#import uobrainflex.utils.image_utilities as im_util
import matplotlib.pyplot as plt
import numpy as np
import tkinter as tk
from tkinter import filedialog

%matplotlib qt
import matplotlib.pyplot as plt

root = tk.Tk()
root.withdraw()

mmm_title = 'Select multi modal map image'
mmm_dir = 'C:\\Users\\admin\\Desktop\\figure dump\\CCF'
mmm_image_path = filedialog.askopenfilename(title = mmm_title, initialdir = mmm_dir)

##hardcode this? or put it in conf file?
ccf_title = 'Select CCF image'
ccf_dir = 'C:\\Users\\admin\\Desktop\\figure dump\\CCF'
ccf_file_path = filedialog.askopenfilename(title = ccf_title, initialdir = ccf_dir)


save_folder = os.path.dirname(mmm_image_path)
subject = os.path.basename(mmm_image_path)
subject = subject[:subject.find("_")]


mmm_image = io.imread(mmm_image_path)
ccf_image = io.imread(ccf_file_path)
mmm_points, ccf_points, figure = set_transform_anchors(mmm_image, ccf_image)

plt.savefig(save_folder + '/' + subject +'_mmm_ccf_fit.png')
np.save(save_folder + '/' + subject + '_mmm_points',mmm_points)
np.save(save_folder + '/' + subject + '_ccf_points',ccf_points)

plt.savefig(os.path.join(PlotDir,'_mmm_ccf_fit_{}.png'.format(fsuffix)))
np.save(os.path.join(PlotDir,'_mmm_points_{}'.format(fsuffix)),mmm_points)
np.save(os.path.join(PlotDir,'_ccf_points_{}'.format(fsuffix)),ccf_points)

In [34]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 16 17:09:27 2021
@author: Daniel Hulsey
this script aligns a ccf to an individual session, and extracts dff data for each indicated region. 
Running it fully requires:
    1. saved dff trace for the session being analyzed
    2. saved points to align CCF to multimodalmap session
    3. blood vessel images from the CCF and session for analysis for alignment
Save location for the region dff traces needs to be better defined    
"""

from skimage import io
#import uobrainflex.utils.image_utilities as im_util

import numpy as np
import tkinter as tk
from tkinter import filedialog
import glob 
from PIL import Image

%matplotlib qt
import matplotlib.pyplot as plt

def area_mean_dff(dff,pixel_list):
    xs = pixel_list[0,:]
    ys = pixel_list[1,:]
    return np.mean(dff[:,xs,ys],axis=1)

def area_max_dff(dff,pixel_list):
    xs = pixel_list[0,:]
    ys = pixel_list[1,:]
    return np.max(dff[:,xs,ys],axis=1)

# select folders and align session and mmm vessel images
newfolders=''
while newfolders != 'n': # repeat until user indicates that new folders should not be selected
    
    subject_title = 'select subject mmm folder'
    session_title = 'select session folder'
    root = tk.Tk()
    root.withdraw()
    subject_folder = filedialog.askdirectory(title = subject_title)
    session_folder = filedialog.askdirectory(title = session_title)
    save_folder = session_folder
    

    mmm_image_path = glob.glob(subject_folder + '/*MMM.png')[0]
    
    # session_vessels_path = glob.glob(subject_folder + '/*vessel*')[0] ## this is the future one. below for testing
    session_vessels_path = glob.glob(session_folder + '/*MMM.png')[0]
        
    mmm_image = io.imread(mmm_image_path)
    vessel_image = io.imread(session_vessels_path)
    
    repeat_align=''
    while repeat_align != 'n': # repeat until user replys to not
        session_vessel_points, session_mmm_points, figure = set_transform_anchors(vessel_image, mmm_image)
        plt.savefig(save_folder +'\mmm_to_session_align.png')
        repeat_align = input("repeat alignment with these images? Y/n\n")
    newfolders = input("repeat alignment with different folder selections? Y/n\n")
    

plt.savefig(save_folder +'\mmm_to_session_align.png')
np.save(save_folder + '\session_mmm_points',session_mmm_points)
np.save(save_folder + '\session_vessel_points',session_vessel_points)

# # Now grab MMM & CCF alignments & DFF, create session masks, and grab mean region dffs
# print('loading dff data')
# # dff_filepath =  glob.glob(session_folder + '/*dff.npy')[0] # this is real one to use. hardcoded to demo with no real data
# dff_filepath =  'T:/BW048_211013_115911/WF_raw/WF_raw_dff.npy'
# dff = np.load(dff_filepath)

mmm_points_filepath = glob.glob(subject_folder + '/*mmm_points.npy')[0]
ccf_points_filepath = glob.glob(subject_folder + '/*ccf_points.npy')[0]
masks_filepath = 'C:\\Users\\McCormick Lab\\Documents\\Python\\Hulsey_A1V1M2_CCF_affine_Jan1122\\22deg\\masks.npy'
areas_filepath = 'C:\\Users\\McCormick Lab\\Documents\\Python\\Hulsey_A1V1M2_CCF_affine_Jan1122\\22deg\\areas.npy'

mmm_points = np.load(mmm_points_filepath)
ccf_points = np.load(ccf_points_filepath)
masks = np.load(masks_filepath)
areas = np.load(areas_filepath)


# st2, wp2 = set_transform_anchors(session_vessels, mmm_vessels)

print('aligning ccf regions')
session_masks=[]
mmm_masks=[]
for i, mask in enumerate(masks):
    mmm_mask = transform_image(mmm_image, mask, mmm_points, ccf_points)
    mmm_mask[mmm_mask<.001]=0
    mmm_mask[mmm_mask>0]=1
    mmm_masks.append(mmm_mask)
    
    session_mask = transform_image(vessel_image, mmm_mask, session_vessel_points, session_mmm_points)
    session_mask[session_mask<.01]=0
    session_mask[session_mask>0]=1
    
    session_masks.append(session_mask)
session_masks = np.array(session_masks)

mmm_masks = np.array(mmm_masks)

plt.figure()
plt.imshow(np.sum(session_masks,axis=0))

plt.imshow(mmm_mask)
plt.imshow(mask)

session_masks = reduce_mask_overlap(session_masks)

plt.figure()
plt.imshow(np.sum(session_masks,axis=0))

outlines=masks_to_outlines(session_masks)
plt.figure()
plt.imshow(np.sum(outlines,axis=0))

outlines_pil=Image.fromarray(np.sum(outlines,axis=0)*255)
plt.figure()
outlines_pil.show()

plt.figure()
plt.imshow(vessel_image)
plt.imshow(np.sum(outlines,axis=0),cmap="gray")

vessel_outlines=vessel_image
outlines_flat=np.sum(outlines,axis=0)*255

for i in range(len(vessel_image[:,0,0])):
    for j in range(len(vessel_image[0,:,0])):
        if outlines_flat[i,j]>0:
            vessel_outlines[i,j,:]=255

plt.figure()
plt.imshow(vessel_outlines)
plt.savefig(save_folder +'\ccf_to_session_align_meso.tif',dpi=600,format='tif')

vessel_outlines_swap = vessel_outlines.transpose(1, 0, 2)
plt.figure()
plt.imshow(vessel_outlines_swap)
plt.savefig(save_folder +'\ccf_to_session_align_rotated.tif',dpi=600,format='tif')

# print('extracting are dff values')
# area_list = ['r_VISp','r_AUDp','r_MOs']
# for area in area_list:
#     # plt.figure()
#     mask_ind = np.where(areas==area)[0]
#     this_mask = session_masks[mask_ind,:,:][0].T
#     this_dff = area_mean_dff(dff,np.array(np.where(this_mask)))
#     np.save(save_folder + '/' + area + '_dff',this_dff)
    

repeat alignment with these images? Y/n
 n
repeat alignment with different folder selections? Y/n
 n


aligning ccf regions
